<br>

<div align=center><font color=maroon size=6><b>Introduction to modules, layers, and models</b></font></div>

<br>

<font size=4><b>References:</b></font>
1. TF2 Core: <a href="https://www.tensorflow.org/guide" style="text-decoration:none;">TensorFlow Guide</a> 
    * `TensorFlow > Learn > TensorFlow Core > `Guide > <a href="https://www.tensorflow.org/guide/intro_to_modules" style="text-decoration:none;">Introduction to modules, layers, and models</a>
        * Run in <a href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/intro_to_modules.ipynb" style="text-decoration:none;">Google Colab</a>

<br>
<br>
<br>

To do machine learning in TensorFlow, you are likely to need to define, save, and restore a model.

A model is, abstractly: 

* A function that computes something on tensors (a **forward pass**)
* Some variables that can be updated in response to training

In this guide, you will go below the surface of Keras to see how TensorFlow models are defined. This looks at how TensorFlow collects variables and models, as well as how they are saved and restored.

Note: If you instead want to immediately get started with Keras, please see [the collection of Keras guides](https://github.com/tensorflow/docs/blob/master/site/en/guide/keras/).


<br>

In [1]:
import tensorflow as tf

from datetime import datetime

In [2]:
print(tf.__version__)

2.8.0


<br>

In [3]:
%load_ext tensorboard

<br>
<br>
<br>

## Defining models and layers in TensorFlow

Most models are made of layers. Layers are functions with a known mathematical structure that can be reused and have trainable variables.  <font size=4 color=maroon>In TensorFlow, most high-level implementations of layers and models, such as Keras or [Sonnet](https://github.com/deepmind/sonnet), are built on the same foundational class: `tf.Module`.</font>

Here's an example of a very simple `tf.Module` that operates on a scalar tensor:

In [4]:
class SimpleModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.a_variable = tf.Variable(5.0, name="train_me")
        self.non_trainable_variable = tf.Variable(5.0, trainable=False, name="do_not_train_me")
    
    def __call__(self, x):
        return self.a_variable * x + self.non_trainable_variable


simple_module = SimpleModule(name="simple")

simple_module(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

<br>

<font size=3 color=maroon>Modules and, by extension, layers are deep-learning terminology for "objects": they have internal state, and methods that use that state.

There is nothing special about `__call__` except to act like a [Python callable](https://stackoverflow.com/questions/111234/what-is-a-callable); you can invoke your models with whatever functions you wish.</font>

You can set the trainability of variables on and off for any reason, including freezing layers and variables during fine-tuning.

<font size=3 color=maroon>Note: `tf.Module` is the base class for both `tf.keras.layers.Layer` and `tf.keras.Model`, so everything you come across here also applies in Keras.  **For historical compatibility reasons Keras layers do not collect variables from modules, so your models should use only modules or only Keras layers.  However, the methods shown below for inspecting variables are the same in either case.**

By subclassing `tf.Module`, any `tf.Variable` or `tf.Module` instances assigned to this object's properties are automatically collected.  This allows you to save and load variables, and also create collections of `tf.Module`s.</font>

In [5]:
# All trainable variables
print("trainable variables:", simple_module.trainable_variables)

# Every variable
print("all variables:", simple_module.variables)

trainable variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
all variables: (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>, <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)


<br>

This is an example of a two-layer linear layer model made out of modules.

First a dense (linear) layer:

In [7]:
class Dense(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        
        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')
        
    
    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        
        return tf.nn.relu(y)

<br>

And then the complete model, which makes two layer instances and applies them:

In [8]:
class SequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = Dense(in_features=3, out_features=3)
        self.dense_2 = Dense(in_features=3, out_features=2)

    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

# You have made a model!
my_model = SequentialModule(name="the_model")

# Call it, with random results
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[2.2607422 3.5774155]], shape=(1, 2), dtype=float32)


<br>

`tf.Module` instances will automatically collect, recursively, any `tf.Variable` or `tf.Module` instances assigned to it. This allows you to manage collections of `tf.Module`s with a single model instance, and save and load whole models.

In [9]:
print("Submodules:", my_model.submodules)

Submodules: (<__main__.Dense object at 0x00000223178774C0>, <__main__.Dense object at 0x0000022317877040>)


In [10]:
for var in my_model.variables:
    print(var, "\n")

<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[-0.01780389,  1.5158873 , -0.9677137 ],
       [ 0.02213093, -0.37829304, -0.0864052 ],
       [ 0.37461364,  0.6056463 , -0.6223783 ]], dtype=float32)> 

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[ 0.8449049 , -0.16288586],
       [ 0.46495926,  1.0615283 ],
       [ 1.0744289 ,  0.07425021]], dtype=float32)> 



<br>
<br>

### Waiting to create variables

You may have noticed here that you have to define both input and output sizes to the layer.  This is so the `w` variable has a known shape and can be allocated.

<font size=3 color=maroon>By deferring variable creation to the first time the module is called with a specific input shape, you do not need specify the input size up front.</font>

In [11]:
class FlexibleDenseModule(tf.Module):
    # Note: No need for `in_features`
    def __init__(self, out_features, name=None):
        super().__init__(name=name)
        self.is_built = False
        self.out_features = out_features

    def __call__(self, x):
        # Create variables on first call.
        if not self.is_built:
            self.w = tf.Variable(tf.random.normal([x.shape[-1], self.out_features]), name='w')
            self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
            self.is_built = True

        y = tf.matmul(x, self.w) + self.b
        
        return tf.nn.relu(y)

In [12]:
# Used in a module
class MySequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = FlexibleDenseModule(out_features=3)
        self.dense_2 = FlexibleDenseModule(out_features=2)

    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_model = MySequentialModule(name="the_model")
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([[0.         0.27327514]], shape=(1, 2), dtype=float32)


<br>

<font size=3 color=maroon>This flexibility is why TensorFlow layers often only need to specify the shape of their outputs, such as in `tf.keras.layers.Dense`, rather than both  the input and output size.</font>

<br>

## Saving weights

<font size=3 color=maroon>You can save a `tf.Module` as both a [checkpoint](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb) and a [SavedModel](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/saved_model.ipynb).</font>

<br>

<font size=3 color=maroon>Checkpoints are just the weights (that is, the values of the set of variables inside the module and its submodules):</font>

In [13]:
chkp_path = "save_model/1-5_ckpt/my_checkpoint"    # 1-5_ckpt 是文件夹，my_checkpoint 是各不同后缀的文件名
checkpoint = tf.train.Checkpoint(model=my_model)
checkpoint.write(chkp_path)

'save_model/1-5_ckpt/my_checkpoint'

<br>

<font size=3 color=maroon>Checkpoints consist of two kinds of files: **the data itself** and **an index file for metadata**. 
* The ***index file*** keeps track of what is actually saved and the numbering of checkpoints, 
* while the ***checkpoint data*** contains the variable values and their attribute lookup paths.
</font>

In [14]:
!ls save_model/1-5_ckpt/my_checkpoint*

save_model/1-5_ckpt/my_checkpoint.data-00000-of-00001
save_model/1-5_ckpt/my_checkpoint.index


<br>

<font size=3 color=maroon>You can look inside a checkpoint to be sure the whole collection of variables is saved, sorted by the **Python object** that contains them.</font>

In [15]:
tf.train.list_variables(chkp_path)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/dense_1/b/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('model/dense_1/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 3]),
 ('model/dense_2/b/.ATTRIBUTES/VARIABLE_VALUE', [2]),
 ('model/dense_2/w/.ATTRIBUTES/VARIABLE_VALUE', [3, 2])]

<br>

<font size=3 color=maroon>During distributed (multi-machine) training they can be sharded,  which is why they are numbered (e.g., '00000-of-00001').  In this case, though, there is only have one shard.

When you load models back in, you overwrite the values in your Python object.</font>

In [16]:
new_model = MySequentialModule()
new_checkpoint = tf.train.Checkpoint(model=new_model)
new_checkpoint.restore("save_model/1-5_ckpt/my_checkpoint")

# Should be the same result as above
new_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.        , 0.27327514]], dtype=float32)>

<br>

<font size=3 color=maroon>**Note**: As checkpoints are at the heart of long training workflows `tf.checkpoint.CheckpointManager` is a helper class that makes checkpoint management much easier. Refer to the [Training checkpoints guide](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb) for more details.</font>

<br>
<br>
<br>

## Saving functions

<font size=3 color=maroon>TensorFlow can run models without the original Python objects, as demonstrated by [TensorFlow Serving](https://tensorflow.org/tfx) and [TensorFlow Lite](https://tensorflow.org/lite), even when you download a trained model from [TensorFlow Hub](https://tensorflow.org/hub).

TensorFlow needs to know how to do the computations described in Python, but **without the original code**. To do this, you can make a **graph**, which is described in the [Introduction to graphs and functions guide](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/intro_to_graphs.ipynb).

This graph contains operations, or *ops*, that implement the function.

You can define a graph in the model above by adding the `@tf.function` decorator to indicate that this code should run as a graph.</font>

In [17]:
class MySequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)

        self.dense_1 = Dense(in_features=3, out_features=3)
        self.dense_2 = Dense(in_features=3, out_features=2)

    @tf.function
    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

# You have made a model with a graph!
my_model = MySequentialModule(name="the_model")

<br>

The module you have made works exactly the same as before.  Each unique signature passed into the function creates a separate graph. Check the [Introduction to graphs and functions guide](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/intro_to_graphs.ipynb) for details.

In [18]:
print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[[0. 0.]
  [0. 0.]]], shape=(1, 2, 2), dtype=float32)


<br>

<font size=3 color=maroon>You can visualize the graph by tracing it within a TensorBoard summary.</font>

In [19]:
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/1-5_%s" % stamp                # 文件夹名
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = MySequentialModule()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True)
tf.profiler.experimental.start(logdir)

# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
    tf.summary.trace_export(name="my_func_trace",
                            step=0,
                            profiler_outdir=logdir)

tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)


<br>

Launch TensorBoard to view the resulting trace:

In [20]:
#docs_infra: no_execute
%tensorboard --logdir logs/func

<br>

```
===============================================================================================================
```

![A screenshot of the graph in TensorBoard](images/tensorboard_graph.png)

<br>
<br>
<br>

## Creating a `SavedModel`

<font size=3 color=maroon>The recommended way of sharing completely trained models is to use `SavedModel`.  `SavedModel` contains both a collection of functions and a collection of weights.</font>

You can save the model you have just trained as follows:

In [22]:
# help(tf.saved_model.save)

In [24]:
# 1-5_the-save-model 是一个文件夹
tf.saved_model.save(my_model, "save_model/1-5_the-save-model")

INFO:tensorflow:Assets written to: save_model/1-5_the-save-model\assets


In [25]:
# Inspect the SavedModel in the directory
!ls -l save_model/1-5_the-save-model

total 16
drwxr-xr-x 1 18617 197611     0 May 25 11:42 assets
-rw-r--r-- 1 18617 197611 13855 May 25 11:42 saved_model.pb
drwxr-xr-x 1 18617 197611     0 May 25 11:42 variables


In [26]:
# The variables/ directory contains a checkpoint of the variables 
!ls -l save_model/1-5_the-save-model/variables

total 2
-rw-r--r-- 1 18617 197611 456 May 25 11:42 variables.data-00000-of-00001
-rw-r--r-- 1 18617 197611 356 May 25 11:42 variables.index


<br>

<font size=3 color=maroon>The `saved_model.pb` file is a [protocol buffer](https://developers.google.com/protocol-buffers) describing the functional `tf.Graph`.

Models and layers can be **loaded from this representation without actually making an instance of the class that created it**.  This is desired in situations where you do not have (or want) a Python interpreter, such as serving at scale or on an edge device, or in situations where the original Python code is not available or practical to use.</font>

You can load the model as new object:

In [27]:
new_model = tf.saved_model.load("save_model/1-5_the-save-model")

<br>

<font size=4 color=maroon>`new_model`, created from loading a saved model, is an internal TensorFlow user object without any of the class knowledge. It is not of type `SequentialModule`.</font>

In [28]:
isinstance(new_model, SequentialModule)

False

In [29]:
isinstance(new_model, MySequentialModule)

False

<br>

<font size=3 color=maroon>This new model works on the already-defined input signatures. You can't add more signatures to a model restored like this.</font>

In [30]:
print(my_model([[2.0, 2.0, 2.0]]))
print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

tf.Tensor([[0. 0.]], shape=(1, 2), dtype=float32)
tf.Tensor(
[[[0. 0.]
  [0. 0.]]], shape=(1, 2, 2), dtype=float32)


<br>

Thus, using `SavedModel`, you are able to save TensorFlow weights and graphs using `tf.Module`, and then load them again.

<br>
<br>
<br>

## Keras models and layers

Note that up until this point, there is no mention of Keras. You can build your own high-level API on top of `tf.Module`, and people have.  

In this section, you will examine how Keras uses `tf.Module`.  A complete user guide to Keras models can be found in the [Keras guide](https://www.tensorflow.org/guide/keras/sequential_model).


<br>

### Keras layers

<font size=3 color=maroon>`tf.keras.layers.Layer` is the base class of all Keras layers, and it inherits from `tf.Module`.

You can convert a module into a Keras layer just by swapping out the parent and then changing `__call__` to `call`:</font>

In [31]:
class MyDense(tf.keras.layers.Layer):
    # Adding **kwargs to support base Keras layer arguments
    def __init__(self, in_features, out_features, **kwargs):
        super().__init__(**kwargs)

        # This will soon move to the build step; see below
        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')
    
    def call(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

simple_layer = MyDense(name="simple", in_features=3, out_features=3)

<br>

<font size=3 color=maroon>Keras layers have their own `__call__` that does some bookkeeping described in the next section and then calls `call()`. You should notice no change in functionality.</font>

In [32]:
simple_layer([[2.0, 2.0, 2.0]])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[1.8730469, 0.       , 4.642761 ]], dtype=float32)>

<br>
<br>

### The `build` step

<font size=3 color=maroon>As noted, it's convenient in many cases to wait to create variables until you are sure of the input shape.

Keras layers come with an extra lifecycle step that allows you more flexibility in how you define your layers. This is defined in the `build` function.

`build` is called **exactly once**, and it is called with the shape of the input. It's usually used to create variables (weights).</font>

You can rewrite `MyDense` layer above to be flexible to the size of its inputs:


In [33]:
class FlexibleDense(tf.keras.layers.Layer):
    # Note the added `**kwargs`, as Keras supports many arguments
    def __init__(self, out_features, **kwargs):
        super().__init__(**kwargs)
        self.out_features = out_features

    def build(self, input_shape):  # Create the state of the layer (weights)
        self.w = tf.Variable(tf.random.normal([input_shape[-1], self.out_features]), name='w')
        self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

    def call(self, inputs):  # Defines the computation from inputs to outputs
        return tf.matmul(inputs, self.w) + self.b


# Create the instance of the layer
flexible_dense = FlexibleDense(out_features=3)

<br>

<font size=3 color=maroon>At this point, the model has not been built, so there are no variables:</font>

In [34]:
flexible_dense.variables

[]

Calling the function allocates appropriately-sized variables:

In [35]:
# Call it, with predictably random results
print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])))

Model results: tf.Tensor(
[[-3.2404785  3.5917969 -2.7041016]
 [-4.860718   5.3876953 -4.0561523]], shape=(2, 3), dtype=float32)


In [36]:
flexible_dense.variables

[<tf.Variable 'flexible_dense/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[-0.14959137,  0.27173236,  0.9008185 ],
        [-0.92553365,  0.31517097, -1.0899538 ],
        [-0.54538816,  1.2092029 , -1.1628331 ]], dtype=float32)>,
 <tf.Variable 'flexible_dense/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

<br>

<font size=4 color=maroon>Since `build` is only called once, inputs will be rejected if the input shape is not compatible with the layer's variables:</font>

In [37]:
try:
    print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0, 2.0]])))
except tf.errors.InvalidArgumentError as e:
    print("Failed:", e)

Failed: Exception encountered when calling layer "flexible_dense" (type FlexibleDense).

Matrix size-incompatible: In[0]: [1,4], In[1]: [3,3] [Op:MatMul]

Call arguments received:
  • inputs=tf.Tensor(shape=(1, 4), dtype=float32)


<br>

<font size=3 color=maroon>Keras layers have a lot more extra features including:

* Optional losses
* Support for metrics
* Built-in support for an optional `training` argument to differentiate between training and inference use
* `get_config` and `from_config` methods that allow you to accurately store configurations to allow model cloning in Python

Read about them in the [full guide](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/keras/custom_layers_and_models.ipynb) to custom layers and models.</font>

<br>

### Keras models

<font size=3 color=maroon>You can define your model as nested Keras layers.

However, Keras also provides a full-featured model class called `tf.keras.Model`. **It inherits from `tf.keras.layers.Layer`**, so a Keras model can be used, nested, and saved in the same way as Keras layers.<br>
**Keras models come with extra functionality that makes them easy to train, evaluate, load, save, and even train on multiple machines**.</font>

You can define the `SequentialModule` from above with nearly identical code, again converting `__call__` to `call()` and changing the parent:

In [38]:
class MySequentialModel(tf.keras.Model):
    def __init__(self, name=None, **kwargs):
        super().__init__(**kwargs)

        self.dense_1 = FlexibleDense(out_features=3)
        self.dense_2 = FlexibleDense(out_features=2)
    
    def call(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

# You have made a Keras model!
my_sequential_model = MySequentialModel(name="the_model")

# Call it on a tensor, with random results
print("Model results:", my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))


Model results: tf.Tensor([[ -4.349065 -13.326223]], shape=(1, 2), dtype=float32)


<br>

All the same features are available, including tracking variables and submodules.

<font size=3 color=maroon>**Note**: To emphasize the note above, **a raw `tf.Module` nested inside a Keras layer or model will not get its variables collected for training or saving**.  Instead, nest Keras layers inside of Keras layers.</font>

In [39]:
my_sequential_model.variables

[<tf.Variable 'my_sequential_model/flexible_dense_1/w:0' shape=(3, 3) dtype=float32, numpy=
 array([[ 1.2936931 ,  0.04194016,  0.2020025 ],
        [ 0.53545356, -0.27152982, -2.1716425 ],
        [ 2.602264  ,  0.05510617,  2.0047448 ]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_1/b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/w:0' shape=(3, 2) dtype=float32, numpy=
 array([[-0.5049483 , -1.4939554 ],
        [-0.29936025,  0.30080822],
        [ 0.28571782,  0.23290311]], dtype=float32)>,
 <tf.Variable 'my_sequential_model/flexible_dense_2/b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [40]:
my_sequential_model.submodules

(<__main__.FlexibleDense at 0x22317b52df0>,
 <__main__.FlexibleDense at 0x2231319fca0>)

<br>

<font size=3 color=maroon>Overriding `tf.keras.Model` is a very Pythonic approach to building TensorFlow models.  If you are migrating models from other frameworks, this can be very straightforward.

If you are constructing models that are simple assemblages of existing layers and inputs, you can save time and space by using the [functional API](./keras/functional.ipynb), which comes with additional features around model reconstruction and architecture.</font>

Here is the same model with the functional API:

In [41]:
inputs = tf.keras.Input(shape=[3,])

x = FlexibleDense(3)(inputs)
x = FlexibleDense(2)(x)

my_functional_model = tf.keras.Model(inputs=inputs, outputs=x)

my_functional_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 flexible_dense_3 (FlexibleD  (None, 3)                12        
 ense)                                                           
                                                                 
 flexible_dense_4 (FlexibleD  (None, 2)                8         
 ense)                                                           
                                                                 
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


<br>

In [42]:
my_functional_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 2.4913993, -7.3404717]], dtype=float32)>

<br>

<font size=3 color=maroon>The major difference here is that the input shape is specified up front as part of the functional construction process. The `input_shape` argument in this case does not have to be completely specified; you can leave some dimensions as `None`.

**Note**: You do not need to specify `input_shape` or an `InputLayer` in a subclassed model; these arguments and layers will be ignored.</font>

<br>

### Saving Keras models

Keras models can be checkpointed, and that will look the same as `tf.Module`.

Keras models can also be saved with `tf.saved_model.save()`, as they are modules.  However, Keras models have convenience methods and other functionality:

In [43]:
my_sequential_model.save("save_model/1-5_keras-save/exname_of_file")

INFO:tensorflow:Assets written to: save_model/1-5_keras-save/exname_of_file\assets


<br>

Just as easily, they can be loaded back in:

In [44]:
reconstructed_model = tf.keras.models.load_model("save_model/1-5_keras-save/exname_of_file")

<br>

<font size=3 color=maroon>Keras `SavedModels` also save metric, loss, and optimizer states.</font>

This reconstructed model can be used and will produce the same result when called on the same data:

In [45]:
reconstructed_model(tf.constant([[2.0, 2.0, 2.0]]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ -4.349065, -13.326223]], dtype=float32)>

<br>

There is more to know about saving and serialization of Keras models, including providing configuration methods for custom layers for feature support. Check out the [guide to saving and serialization](https://www.tensorflow.org/guide/keras/save_and_serialize).

<br>
<br>
<br>

## What's next

If you want to know more details about Keras, you can follow the existing Keras guides [here](https://github.com/tensorflow/docs/blob/master/site/en/guide/keras/).

Another example of a high-level API built on `tf.module` is Sonnet from DeepMind, which is covered on [their site](https://github.com/deepmind/sonnet).

<br>
<br>
<br>

**Copyright 2020 The TensorFlow Authors.**

```python
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

<br>

<br>
<br>
<br>